In [40]:
import pickle
import numpy as np
import bcolz
import openke
# from openke.config import Trainer, Tester
# from openke.module.model import TransE
# from openke.module.loss import MarginLoss
# from openke.module.strategy import NegativeSampling
from openke.data import TrainDataLoader, TestDataLoader
import json
import collections
from scipy.sparse import csc_matrix
import scipy

In [2]:
train_dataloader = TrainDataLoader(
	in_path = "./benchmarks/FB60K/",
	nbatches = 100,
	threads = 8,
	sampling_mode = "normal",
	bern_flag = 1,
	filter_flag = 1,
	neg_ent = 25,
	neg_rel = 0)
entity_size = train_dataloader.get_ent_tot()

In [3]:
entity_size

69515

In [4]:
base_path = '/home/ubuntu/text-pwrd-kg-reasoning/OpenKE/benchmarks/FB60K/'
entity2id = {}
with open("/home/ubuntu/text-pwrd-kg-reasoning/OpenKE/benchmarks/FB60K/entity2id.txt") as f:
    line = f.readline().split()
    max_id = int(line[0])
    line = f.readline().split()
    while line:
        entity2id[line[0]] = int(line[1])
        line = f.readline().split()

In [5]:
with open(base_path + 'train.json') as f:
    train_file = json.load(f) 

In [59]:
similarity_matrix = collections.Counter()
for data in train_file:
#     head_id = "/" + "/".join(data['head']['id'].split("."))
    head_entity =  data['head']['id']
    tail_entity = data['tail']['id']
    head_idx = entity2id[head_entity]
    tail_idx = entity2id[tail_entity]
    if head_idx > tail_idx:
        tmp = head_idx
        head_idx = tail_idx
        tail_idx = tmp
    fin_key = str(head_idx) + '_' + str(tail_idx)                                                                                      
    similarity_matrix[fin_key] += 1
similarity_matrix = dict(similarity_matrix)

In [63]:
row = []
col = []
data = []
for k,v in similarity_matrix.items():
    k = k.split("_")
    i = int(k[0])
    j = int(k[1])
    row.append(i)
    col.append(j)
    data.append(v)

In [64]:
sim_matrix = csc_matrix((data, (row, col)), shape=(entity_size, entity_size))

In [65]:
scipy.sparse.save_npz(base_path + 'sparse_matrix.npz', sim_matrix)
sparse_matrix = scipy.sparse.load_npz(base_path + 'sparse_matrix.npz')